# **Crypto forecasting tutorial**

# G-Research Crypto 預測比賽

在 [G-Research Crypto 預測競賽]（鏈接）中，參與者面臨著預測一系列主要加密貨幣的價格回報的挑戰。 為了方便您參與，我們創建了本教程筆記本，涵蓋了加密預測挑戰的一些相關概念。

該筆記本介紹了加密預測，描述了數據集的結構和元素、一些相關的統計屬性，以及構建了幾個 ML 基線模型並提供了示例代碼提交。



## 加密貨幣市場

首先，快速介紹加密世界。 加密貨幣已經成為一個非常受歡迎和動蕩的市場，為投資者帶來巨額回報（以及損失）。 已經創建了數以千計的加密貨幣，其中包括比特幣 (BTC)、以太幣 (ETH) 或狗狗幣 (DOGE) 在內的一些主要加密貨幣，你們中的許多人都聽說過。

根據 CryptoCompare（截至 2021 年 7 月 25 日）的數據，加密貨幣在加密貨幣交易所廣泛交易，去年平均每天交易量為 410 億美元。

不同加密貨幣之間的價格變化是高度相互關聯的。 例如，比特幣歷來是加密貨幣價格變化的主要驅動力，但其他代幣也會影響市場。
## 加密貨幣市場

首先，快速介紹加密世界。 加密貨幣已經成為一個非常受歡迎和動蕩的市場，為投資者帶來巨額回報（以及損失）。 已經創建了數以千計的加密貨幣，其中包括比特幣 (BTC)、以太幣 (ETH) 或狗狗幣 (DOGE) 在內的一些主要加密貨幣，你們中的許多人都聽說過。

根據 CryptoCompare（截至 2021 年 7 月 25 日）的數據，加密貨幣在加密貨幣交易所廣泛交易，去年平均每天交易量為 410 億美元。

不同加密貨幣之間的價格變化是高度相互關聯的。 例如，比特幣歷來是加密貨幣價格變化的主要驅動力，但其他代幣也會影響市場。


## 預測回報

金融建模的一項基本任務是預測價格在不久的將來會如何表現。使用歷史價格的時間序列作為訓練數據，我們想要預測價格會上漲還是下跌，以及上漲多少，即資產*回報*。

在本次比賽中，Kagglers 的挑戰是建立機器學習模型來預測 14 種流行加密貨幣的回報，時間範圍從幾分鐘到幾小時。您將可以訪問數百萬行每分鐘的加密貨幣交易數據，您將使用這些數據同時為所有 14 種資產設計預測模型。您的預測將根據它們與比賽結束後三個月評估期內收集的真實市場數據的相關程度進行評估。

加密貨幣收益預測仍然是一項開放且極具挑戰性的預測任務。鑑於資產的極端波動性、數據的非平穩性、市場和 meme 操縱、資產之間的相關性和快速變化的市場條件，這對於 ML 社區來說是一個引人入勝的問題領域。我們希望您和我們一樣發現它令人著迷！


# 數據集描述
​​​
現在，讓我們深入研究數據！ 我們首先加載比賽的數據集並檢查其基本屬性。

## 加載訓練集

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
data_folder = "../input/g-research-crypto-forecasting/"
!ls $data_folder

In [ ]:
crypto_df = pd.read_csv(data_folder + 'train.csv')

In [ ]:
crypto_df.head(10)

We can see that each row of the data set has the trading data for an asset, at a given minute timestamp, described in detail below. 

## 數據特徵
我們可以看到數據集中包含的不同特徵。具體來說，每個資產包含的功能如下：
* **timestamp**：所有時間戳都作為第二個 Unix 時間戳返回（自 1970-01-01 00:00:00.000 UTC 以來經過的秒數）。此數據集中的時間戳是 60 的倍數，表示每分鐘的數據。
* **Asset_ID**：對應於其中一種加密貨幣的資產 ID（例如，`Asset_ID = 1` 用於比特幣）。從“Asset_ID”到加密資產的映射包含在“asset_details.csv”中。
* **Count**：時間間隔內的交易總數（最後一分鐘）。
* **Open**：時間間隔的開盤價（美元）。
* **High**：在時間間隔內達到的最高價格（以美元計）。
* **Low**：在時間間隔內達到的最低價格（以美元計）。
* **Close**：時間間隔的收盤價（美元）。
* **Volume**：買入或賣出的資產數量，以基礎貨幣美元顯示。
* **VWAP**：資產在時間間隔內的平均價格，按交易量加權。 VWAP 是貿易數據的聚合形式。
* **Target**：資產在 15 分鐘內的剩餘對數回報。

前兩列定義此數據行的時間和資產索引。中間的 6 列是特徵列，其中包含該資產的交易數據和分鐘。最後一列是預測目標，我們稍後會更詳細地介紹。

我們還查看資產信息，包括所有資產的列表、“Asset_ID”到資產的映射，以及用於衡量其在評估指標中的相對重要性的每個資產的權重​​。

In [ ]:
asset_details = pd.read_csv(data_folder + 'asset_details.csv')
asset_details

##燭台圖

交易數據格式是市場數據的聚合形式，包括開盤價、最高價、最低價和收盤價。 我們可以通過常用的燭台條形圖將這些數據可視化，允許交易者對盤中值進行技術分析。 柱體長度代表當天交易的開盤價和收盤價之間的價格範圍。 當柱為紅色時，表示收盤價低於開盤價，否則為綠色。 這些也被稱為看漲和看跌燭台。 條形圖上方和下方的燈芯顯示該區間交易的最高價和最低價。

我們可以使用 `plotly` 庫可視化比特幣價格的一部分。 圖的底部顯示了一個範圍滑塊，您可以使用它來放大圖。

In [ ]:
btc = crypto_df[crypto_df["Asset_ID"]==1].set_index("timestamp") # Asset_ID = 1 for Bitcoin
btc_mini = btc.iloc[-200:] # Select recent data rows

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(x=btc_mini.index, open=btc_mini['Open'], high=btc_mini['High'], low=btc_mini['Low'], close=btc_mini['Close'])])
fig.show()

# 預處理

## 處理缺失數據


讓我們檢查另一個重要資產以太坊的數據。

In [ ]:
eth = crypto_df[crypto_df["Asset_ID"]==6].set_index("timestamp") # Asset_ID = 6 for Ethereum
eth.info(show_counts =True)

我們可以看到訓練集中的行數，以及目標列的缺失值，我們將在稍後解決。 讓我們確認一下：


In [ ]:
eth.isna().sum()

讓我們使用從時間戳到“日期時間”的轉換來檢查比特幣和以太坊數據的`datetime`。.

In [ ]:
btc.head()

In [ ]:
beg_btc = btc.index[0].astype('datetime64[s]')
end_btc = btc.index[-1].astype('datetime64[s]')
beg_eth = eth.index[0].astype('datetime64[s]')
end_eth = eth.index[-1].astype('datetime64[s]')

print('BTC data goes from ', beg_btc, 'to ', end_btc)
print('Ethereum data goes from ', beg_eth, 'to ', end_eth)

對於給定的一分鐘，缺少的資產數據不是由 NaN 表示，而是由這些行的缺失表示。 我們可以檢查連續行之間的時間戳差異，看看是否有缺失數據。

In [ ]:
(eth.index[1:]-eth.index[:-1]).value_counts().head()

請注意，數據中有許多空白。 要使用大多數時間序列模型，我們應該將數據預處理為沒有時間間隔的格式。 為了填補空白，我們可以使用 .reindex() 方法進行前向填充，用之前的有效值填補空白。

In [ ]:
eth = eth.reindex(range(eth.index[0],eth.index[-1]+60,60),method='pad')

並檢查現在沒有時間間隔。

In [ ]:
(eth.index[1:]-eth.index[:-1]).value_counts().head()

## 數據可視化
​​​
我們將從可視化我們選擇的兩種資產的收盤價開始。

In [ ]:
import matplotlib.pyplot as plt

# 繪製兩個選定資產的 vwap 時間序列
f = plt.figure(figsize=(15,4))

# 填充 BTC 的缺失值
btc = btc.reindex(range(btc.index[0],btc.index[-1]+60,60),method='pad')

ax = f.add_subplot(121)
plt.plot(btc['Close'], label='BTC')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Bitcoin')

ax2 = f.add_subplot(122)
ax2.plot(eth['Close'], color='red', label='ETH')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Ethereum')

plt.tight_layout()
plt.show()

這些資產的歷史完全不同，但我們可以檢查它們最近是否相關。

In [ ]:
import time

# auxiliary function, from datetime to timestamp
totimestamp = lambda s: np.int32(time.mktime(datetime.strptime(s, "%d/%m/%Y").timetuple()))

# create intervals
btc_mini_2021 = btc.loc[totimestamp('01/06/2021'):totimestamp('01/07/2021')]
eth_mini_2021 = eth.loc[totimestamp('01/06/2021'):totimestamp('01/07/2021')]

In [ ]:
# plot time series for both chosen assets
f = plt.figure(figsize=(7,8))

ax = f.add_subplot(211)
plt.plot(btc_mini_2021['Close'], label='btc')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Bitcoin Close')

ax2 = f.add_subplot(212)
ax2.plot(eth_mini_2021['Close'], color='red', label='eth')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Ethereum Close')

plt.tight_layout()
plt.show()

On shorter intervals we can visually see some potential correlation between both assets, with some simultaneous ups and downs. A better format for analyzing such movements is by calculating asset returns. 

## 日誌返回
​​​
為了分析資產的價格變化，我們可以處理價格差異。 然而，不同的資產表現出不同的價格尺度，因此它們的回報不易比較。 我們可以通過計算價格的百分比變化來解決這個問題，也稱為回報。 這一回報與我們投資資本的百分比變化相吻合。
​​​
回報在金融中被廣泛使用，但是對數回報更適合時間序列的數學建模，因為它們是隨時間相加的。 此外，雖然常規回報率不能低於 -100%，但對數回報率不受限制。
​​​
為了計算對數回報，我們可以簡單地取兩個連續價格之間比率的對數。 第一行將有一個空返回，因為先前的值是未知的，因此空返回數據點將被刪除。## Log returns

In order to analyze price changes for an asset we can deal with the price difference. However, different assets exhibit different price scales, so that the their returns are not readily comparable. We can solve this problem by computing the percentage change in price instead, also known as the return. This return coincides with the percentage change in our invested capital.

Returns are widely used in finance, however log returns are preferred for mathematical modelling of time series, as they are additive across time. Also, while regular returns cannot go below -100%, log returns are not bounded.

To compute the log return, we can simply take the logarithm of the ratio between two consecutive prices. The first row will have an empty return as the previous value is unknown, therefore the empty return data point will be dropped.

In [ ]:
# define function to compute log returns
def log_return(series, periods=1):
    return np.log(series).diff(periods=periods)

我們可以可視化我們的兩個資產的日誌回報。 看看信號現在看起來更像白噪聲，與價格的時間序列相比，漂移更小。

In [ ]:
import scipy.stats as stats

lret_btc = log_return(btc_mini_2021.Close)[1:]
lret_eth = log_return(eth_mini_2021.Close)[1:]
lret_btc.rename('lret_btc', inplace=True)
lret_eth.rename('lret_eth', inplace=True)

plt.figure(figsize=(8,4))
plt.plot(lret_btc);
plt.plot(lret_eth);
plt.show()

##  資產之間的相關性

我們之前假設加密資產回報可能表現出一些相關性。 現在讓我們更詳細地檢查一下。

我們可以檢查在我們選擇的 2021 年期間，比特幣和以太坊之間的相關性如何隨時間變化。

In [ ]:
# join two asset in single DataFrame

lret_btc_long = log_return(btc.Close)[1:]
lret_eth_long = log_return(eth.Close)[1:]
lret_btc_long.rename('lret_btc', inplace=True)
lret_eth_long.rename('lret_eth', inplace=True)
two_assets = pd.concat([lret_btc_long, lret_eth_long], axis=1)

# group consecutive rows and use .corr() for correlation between columns
corr_time = two_assets.groupby(two_assets.index//(10000*60)).corr().loc[:,"lret_btc"].loc[:,"lret_eth"]

corr_time.plot();
plt.xticks([])
plt.ylabel("Correlation")
plt.title("Correlation between BTC and ETH over time");

請注意資產之間的高但可變的相關性。 在這裡，我們可以看到隨著時間的推移存在一些變化的動態，這對於這個時間序列挑戰至關重要，即如何在高度非平穩的環境中執行預測。

系統或過程的靜止行為的特徵在於不隨時間變化的統計特性，例如均值、方差和自相關。 另一方面，非平穩行為的特徵是統計特性隨時間的連續變化。 平穩性很重要，因為許多有用的分析工具、統計測試和模型都依賴於它。

我們還可以通過可視化相關矩陣來檢查所有資產之間的相關性。 請注意，某些資產的成對相關性如何高於其他資產。

In [ ]:
# create dataframe with returns for all assets
all_assets_2021 = pd.DataFrame([])
for asset_id, asset_name in zip(asset_details.Asset_ID, asset_details.Asset_Name):
  asset = crypto_df[crypto_df["Asset_ID"]==asset_id].set_index("timestamp")
  asset = asset.loc[totimestamp('01/01/2021'):totimestamp('01/05/2021')]
  asset = asset.reindex(range(asset.index[0],asset.index[-1]+60,60),method='pad')
  lret = log_return(asset.Close.fillna(0))[1:]
  all_assets_2021 = all_assets_2021.join(lret, rsuffix=asset_name, how="outer")

In [ ]:
asset_details.Asset_ID, asset_details.Asset_Name

In [ ]:
plt.imshow(all_assets_2021.corr());
plt.yticks(range(14), asset_details.Asset_Name.values);
plt.xticks(range(14), asset_details.Asset_Name.values, rotation='vertical');
plt.colorbar();

We encourage participants to perform additional statistical analyses to have a stronger grasp on the dataset, including autocorrelation, time-series decomposition and stationarity tests.

# 構建你的預測模型

## 預測目標和評估

該預測競賽旨在預測每項資產 $a$ 價格 $P^a$ 的近期回報。 對於數據集中的每一行，我們都包含預測目標“Target”。 `Target` 來自 15 分鐘內的日誌返回 ($R^a$)。

$$R^a(t) = log (P^a(t+16)\ /\ P^a(t+1))$$

加密資產回報高度相關，在很大程度上跟隨整個加密市場。 由於我們想測試您預測單個資產回報的能力，我們執行線性殘差，在創建目標時從單個資產回報中去除市場信號。 更詳細地說，如果 $M(t)$ 是加權平均市場收益，則目標是：

$$M(t) = \frac{\sum_a w^a R^a(t)}{\sum_a w^a}  \\
\beta^a = \frac{\langle M \cdot R^a \rangle}{\langle M^2 \rangle} \\
\text{Target}^a(t) = R^a(t) - \beta^a M(t)$$

其中括號 $\langle .\rangle$ 表示隨時間推移的滾動平均值（3750 分鐘窗口），並且用於評估指標的相同資產權重 $w^a$。

由於未來價格中的缺失值，某些行的目標值為空。 出於評分目的，測試集基本事實中具有空值的行將被忽略。

在比賽中，您的預測將根據 Pearson 相關係數的加權版本進行評估，權重由資產詳細信息文件中的“權重”列給出。

在本教程中，我們將簡化事物並使用相關性（無權重）進行評估，並僅考慮兩種資產，BTC 和 ETH。

## 功能設計

我們首先設計一些相關的特徵來輸入我們的模型。

In [ ]:
# Select some input features from the trading data: 
# 5 min log return, abs(5 min log return), upper shadow, and lower shadow.
upper_shadow = lambda asset: asset.High - np.maximum(asset.Close,asset.Open)
lower_shadow = lambda asset: np.minimum(asset.Close,asset.Open)- asset.Low

X_btc = pd.concat([log_return(btc.VWAP,periods=5), log_return(btc.VWAP,periods=1).abs(), 
               upper_shadow(btc), lower_shadow(btc)], axis=1)
y_btc = btc.Target

X_eth = pd.concat([log_return(eth.VWAP,periods=5), log_return(eth.VWAP,periods=1).abs(), 
               upper_shadow(eth), lower_shadow(eth)], axis=1)
y_eth = eth.Target

## 為構建預測模型準備數據

由於我們將訓練線性回歸參數，我們需要將訓練集和測試集分開。 為此，我們將計算 X 和 y 並將這些數據拆分為訓練和測試拆分。 請注意，測試拆分代表數據的後面部分，因為它通常在時間序列中完成。

In [ ]:
# select training and test periods
train_window = [totimestamp("01/05/2021"), totimestamp("30/05/2021")]
test_window = [totimestamp("01/06/2021"), totimestamp("30/06/2021")]

# divide data into train and test, compute X and y
# we aim to build simple regression models using a window_size of 1
X_btc_train = X_btc.loc[train_window[0]:train_window[1]].fillna(0).to_numpy()  # filling NaN's with zeros
y_btc_train = y_btc.loc[train_window[0]:train_window[1]].fillna(0).to_numpy()  

X_btc_test = X_btc.loc[test_window[0]:test_window[1]].fillna(0).to_numpy() 
y_btc_test = y_btc.loc[test_window[0]:test_window[1]].fillna(0).to_numpy() 

X_eth_train = X_eth.loc[train_window[0]:train_window[1]].fillna(0).to_numpy()  
y_eth_train = y_eth.loc[train_window[0]:train_window[1]].fillna(0).to_numpy()  

X_eth_test = X_eth.loc[test_window[0]:test_window[1]].fillna(0).to_numpy() 
y_eth_test = y_eth.loc[test_window[0]:test_window[1]].fillna(0).to_numpy() 

我們現在標準化輸入數據。 標準化是將不同變量放在同一尺度上的過程。 在回歸分析中，標準化自變量通常至關重要，否則您可能會冒著獲得誤導性結果的風險。

In [ ]:
from sklearn.preprocessing import StandardScaler
# simple preprocessing of the data 
scaler = StandardScaler()

X_btc_train_scaled = scaler.fit_transform(X_btc_train)
X_btc_test_scaled = scaler.transform(X_btc_test)

X_eth_train_scaled = scaler.fit_transform(X_eth_train)
X_eth_test_scaled = scaler.transform(X_eth_test)

## 基線模型：線性回歸

我們將對我們設計的特徵嘗試一個簡單的線性回歸模型。 請注意，線性回歸在時間序列分析中並不常用，特別是只有一個時間步！

我們比較了兩個線性回歸基線，一個獨立考慮每個資產，一個將所有資產建模在一起的多個輸入。


In [ ]:
from sklearn.linear_model import LinearRegression

# implement basic ML baseline (one per asset)
lr = LinearRegression()
lr.fit(X_btc_train_scaled,y_btc_train)
y_pred_lr_btc = lr.predict(X_btc_test_scaled)

lr.fit(X_eth_train_scaled,y_eth_train)
y_pred_lr_eth = lr.predict(X_eth_test_scaled)

## 評估基線

競爭性能指標是加權相關性。 但是，現在我們將使用簡單的相關性來評估所構建的兩個基線模型。

In [ ]:
# implement more complex baseline (multiple input/output regression model)
from sklearn.multioutput import MultiOutputRegressor

# we concatenate X and y for both assets
X_both_train = np.concatenate((X_btc_train_scaled, X_eth_train_scaled), axis=1)
X_both_test = np.concatenate((X_btc_test_scaled, X_eth_test_scaled), axis=1)
y_both_train = np.column_stack((y_btc_train, y_eth_train))
y_both_test = np.column_stack((y_btc_test, y_eth_test))

# define the direct multioutput model and fit it
mlr = MultiOutputRegressor(LinearRegression())
mlr.fit(X_both_train,y_both_train)
y_pred_lr_both = mlr.predict(X_both_test)

In [ ]:
print('Test score for LR baseline: BTC', f"{np.corrcoef(y_pred_lr_btc, y_btc_test)[0,1]:.2f}", 
                                ', ETH', f"{np.corrcoef(y_pred_lr_eth, y_eth_test)[0,1]:.2f}")
print('Test score for multiple output LR baseline: BTC', f"{np.corrcoef(y_pred_lr_both[:,0], y_btc_test)[0,1]:.2f}", 
                                                ', ETH', f"{np.corrcoef(y_pred_lr_both[:,1], y_eth_test)[0,1]:.2f}")

我們可以看到，對於選擇的訓練和測試期，多資產 LR 模型比簡單地單獨建模每個資產的性能更好。 請注意，由於數據非常不穩定，因此這些結果在不同時期可能會有很大差異。

## 提交

請注意，這是一場代碼競賽，您必須提交您的筆記本才能針對隱藏的私人數據運行。 你的 notebook 應該使用提供的 python 時間序列 API，它可以確保模型不會及時向前看。 使用API請按照[代碼競賽API詳細說明](https://www.kaggle.com/sohier/detailed-api-introduction) 和[基本提交模板](https://www.kaggle.com/sohier/basic-submission-template)。
